In [601]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']
def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def get_f1accs(df: Union[pd.DataFrame, List[pd.DataFrame]]):
    metrics = ['Vanilla test acc','Vanilla test f1','Shadow test acc','Shadow test f1','MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Vanilla runtime per']
    if isinstance(df,list):
        res = []
        for i in df:
            res.append(i[metrics].max())
        return res
    else:
        return df[metrics].max()
    

total_parms_new.columns

Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

In [602]:
def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==False)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==False)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

In [603]:
clean_adam = total_parms[(total_parms['Dp']==False)&(total_parms['Rdp']==False)&(total_parms['Ldp']==False)&(total_parms['Optim type']=='adam')]
clean_sgd = total_parms[(total_parms['Dp']==False)&(total_parms['Rdp']==False)&(total_parms['Ldp']==False)&(total_parms['Optim type']=='sgd')]


In [604]:
clean_sgd

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type


In [605]:
_,clean_sgd = baseline(total_parms_new)

In [606]:
clean_sgd

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type
510,cora,0.8933,0.5738,0.8933,0.7800,0.8644,0.7800,0.8916,0.6144,0.8916,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
511,cora,0.9031,0.6010,0.9031,0.8013,0.7987,0.8013,0.8916,0.5872,0.8916,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
512,cora,0.8785,0.6489,0.8785,0.7931,0.8245,0.7931,0.8736,0.6153,0.8736,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
513,cora,0.8818,0.6372,0.8818,0.7898,0.8272,0.7898,0.9015,0.5717,0.9015,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
514,cora,0.8851,0.6160,0.8851,0.8079,0.8025,0.8079,0.8785,0.6212,0.8785,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,github,0.8576,0.3517,0.8576,0.8520,0.3686,0.8520,0.8527,0.3649,0.8527,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
746,github,0.8592,0.3518,0.8592,0.8531,0.3697,0.8531,0.8518,0.3682,0.8518,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
747,github,0.8621,0.3483,0.8621,0.8531,0.3627,0.8531,0.8471,0.3733,0.8471,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
748,github,0.8535,0.3496,0.8535,0.8582,0.3594,0.8582,0.8553,0.3607,0.8553,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd


In [607]:
clean_list = get_datasetwise_df(clean_sgd)

clean_res_dfs = get_f1accs(clean_list)
clean_res = {}
for i in range(len(dataset_list)):
    clean_res[dataset_list[i]] = clean_res_dfs[i].to_dict()
pd.DataFrame(clean_res).T

,Vanilla test acc,Vanilla test f1,Shadow test acc,Shadow test f1,MIA mlp,MIA svm,MIA ranfor,MIA logi,MIA ada,MIA confidence mse,Vanilla runtime per
cora,0.8112,0.8112,0.8128,0.8128,0.5821,0.5936,0.5665,0.5468,0.5698,0.6026,0.1845
citeseer,0.7219,0.7219,0.7380,0.7380,0.5926,0.5813,0.5739,0.5505,0.5764,0.5952,0.1805
pubmed,0.6105,0.6105,0.5940,0.5940,0.5000,0.5050,0.5187,0.5108,0.5074,0.5123,0.1903
computers,0.8251,0.8251,0.8229,0.8229,0.5130,0.5158,0.5112,0.5193,0.5143,0.5192,0.1857
photo,0.9140,0.9140,0.9198,0.9198,0.5192,0.5192,0.5186,0.5221,0.5180,0.5299,0.1812
cs,0.8312,0.8312,0.8303,0.8303,0.5155,0.5170,0.5093,0.5171,0.5138,0.5139,0.1938
physics,0.9343,0.9343,0.9305,0.9305,0.5103,0.5132,0.5054,0.5129,0.5096,0.5078,0.1977
reddit,0.6953,0.6953,0.6897,0.6897,0.5077,0.5141,0.5081,0.5121,0.5112,0.5131,0.2187
github,0.8582,0.8582,0.8592,0.8592,0.5058,0.5083,0.5086,0.5068,0.5093,0.5107,0.1815
flickr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
